In [1]:
### L6 Numerical Linear Algebra 2
'''
- Direct linear solver methods: LU decomposition/ factorization
- Doolittle's algorithm (constructing L)
- Properties of lower-triangular matrices
- Partial pivoting
'''

# LU decomposition
    # making L and U
    # properties of L
    # partial pivoting
import numpy as np
import scipy.linalg as sl

In [2]:
### LU decomposition/ factorization (direct linear solver)
# multiple RHS vectors with same matrix A
    # can call same code (upper_triangle and back_sub) multiple times but redundant
    # include multiple RHS in Gaussian elimination and back sub on each transformed RHS
    # however RHS vector could depend on earlier RHS vectors

# atomic lower-triangular matrices
    # triangular matrix with diagonals all being 1
    # and off-diagonal enteries all zero apart from one column
    ## inverse are original matrix with signs of off-diagonals changed

In [3]:
## Ex 6.1 lower-triangular matices

# as above: A matrix,
A=np.array([[5., 7. , 5., 9.],
            [5., 14.,   7.,  10.],
            [20., 77., 41., 48.],
            [25., 91.,  55., 67.]])
# lower triangular matrices,
L0 = np.array([[1,0,0,0],
                [-1,1,0,0],
                [-4,0,1,0],
                [-5,0,0,1]])

L1 = np.array([[1,0,0,0],
                [0,1,0,0],
                [0,-7,1,0],
                [0,-8,0,1]])

L2 = np.array([[1,0,0,0],
                [0,1,0,0],
                [0,0,1,0],
                [0,0,-2,1]])

# and their inverse matrices
L0_ = sl.inv(L0)
L1_ = sl.inv(L1)
L2_ = sl.inv(L2)

print("\nMatrix A")
print(A)

print("\nL0")
print(L0)
print(sl.inv(L0))
print("\nL1")
print(L1)
print(sl.inv(L1))
print("\nL2")
print(L2)
print(sl.inv(L2))

# Check the multiplication of arbitrary lower-triangular square matrices is also lower-triangular
print("\nL0 @ L1_")
print(L0 @ L1_)
print("\nL2 @ L1")
print(L2 @ L1)


Matrix A
[[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]

L0
[[ 1  0  0  0]
 [-1  1  0  0]
 [-4  0  1  0]
 [-5  0  0  1]]
[[ 1.  0. -0.  0.]
 [ 1.  1. -0.  0.]
 [ 4.  0.  1.  0.]
 [ 5.  0.  0.  1.]]

L1
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0 -7  1  0]
 [ 0 -8  0  1]]
[[ 1. -0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  7.  1.  0.]
 [ 0.  8.  0.  1.]]

L2
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0 -2  1]]
[[ 1.  0. -0.  0.]
 [ 0.  1. -0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  2.  1.]]

L0 @ L1_
[[ 1.  0.  0.  0.]
 [-1.  1.  0.  0.]
 [-4.  7.  1.  0.]
 [-5.  8.  0.  1.]]

L2 @ L1
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0 -7  1  0]
 [ 0  6 -2  1]]


In [4]:
# Upper matrix = L matrixes @ A
" ORDER MATTERS THESE ARE MATRICES"
" L0 L2 L3 are negative"
" L is DEFINED as = L0_ @ L1_ @ L2_ and (if orig matrix is positive) is positive"
print("\nMatrix A")
print(A)

U = L2 @ (L1 @ (L0 @ A))

print("L2(L1(L0 A))")
print(U)

# A = L matrices inverse @ U
" ORDER INVERSES AS WELL"
A_1 = L0_ @(L1_ @(L2_ @ U))
print("L0_ @(L1_ @(L2_ @ U))")
print(A_1)

L_1 = L0_ @ L1_ @ L2_
print("L0_ @ L1_ @ L2_ = L inv")
print(L_1)

# A = LU
A_2 = L_1 @ U

print("A = L_1 @ U")
print(A_2)

print('\nA = L_1 @ U? ',np.allclose(A, A_2))


Matrix A
[[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]
L2(L1(L0 A))
[[5. 7. 5. 9.]
 [0. 7. 2. 1.]
 [0. 0. 7. 5.]
 [0. 0. 0. 4.]]
L0_ @(L1_ @(L2_ @ U))
[[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]
L0_ @ L1_ @ L2_ = L inv
[[1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [4. 7. 1. 0.]
 [5. 8. 2. 1.]]
A = L_1 @ U
[[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]

A = L_1 @ U?  True


In [5]:
## Ex 6.2 LU decomposition
    # U already constructed in L5 via gaussian elimination
    # entries of L --> Aik/ Akk multipliers (s in L5)
    # save space --> L is lower triangle (multiplier required for diagonal number = that number)


# from L5, modified to exclude b (according to sample sol)
def LU_decomp(A):
    """ A function to covert A into upper triangluar form through row operations.
    The same row operations are performed on the vector b.
    
    Note that this implementation does not use partial pivoting which is introduced below.
    
    Also note that A and b are overwritten, and hence we do not need to return anything
    from the function.
    """
    "n = np.size(b)"
    rows, cols = np.shape(A)
    "np.shape gives tuple"

    # check A is square
    assert(rows == cols)
    # and check A has the same numner of rows as the size of the vector b (columnar)
    "assert(rows == n)"

    L = np.eye(rows)

    # Loop over each pivot row - all but the last row which we will never need to use as a pivot
    " k = row/ column (square matrix)"
    for k in range(rows-1):
        # Loop over each row below the pivot row, including the last row which we do need to update
        for i in range(k+1, rows):
            "for EACH row below pivot row (i = row below pivot)"
            "loop for all rows below pivot to turn column k into 0"
            "k+1 --> after pivot row. start: k+1 end: n"

            # scaling factor for row
            "Row i (below pivot row) column k"
            "k works bc square matrix"
            s = (A[i, k] / A[k, k])

            # update current row A by looping over column j
            "start from k as we assume entries before this are already zero"
            '''for j in range(k, n):
                A[i, j] = A[i, j] - s * A[k, j]'''
            
            A[i, :] = A[i, :] - s * A[k, :]

            # update corresponding b and s
            "b[i] = b[i] - s * b[k]"
            L[i, k] = s

    return L, A

A = np.array([[ 5., 7.,   5.,  9.],
              [ 5., 14.,  7., 10.],
              [20., 77., 41., 48.],
              [25., 91. ,55., 67.]])

A_orig = np.copy(A)

L, U = LU_decomp(A)
# takes only 1 variable

print("A")
print(A_orig)
print("U")
print(U)
print("L")
print(L)

print('\nA = L @ U? ',np.allclose(A_orig, L @ U))



A
[[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]
U
[[5. 7. 5. 9.]
 [0. 7. 2. 1.]
 [0. 0. 7. 5.]
 [0. 0. 0. 4.]]
L
[[1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [4. 7. 1. 0.]
 [5. 8. 2. 1.]]

A = L @ U?  True


In [6]:
### Partial pivoting
# Akk can't be close to zero (s = A[i, k]/ A[k, k])
# swaps rows with one where Aik is largest 
# scipy.linalg's LU decomp uses partial pivoting by default

A = np.array([[ 5., 7.,   5.,  9.],
               [ 5., 14.,  7., 10.],
               [20., 77., 41., 48.],
               [25., 91. ,55., 67.]])

A_orig = np.copy(A)

print("A original")
print(A_orig)

P, L , U = sl.lu(A)
# P = permutation matrix (performs swaps based upon partial pivoting)
    # P @ L @ U = A
    # not identity --> performed row swaps

    # can recorder A in advance in terms of P where P^-1 = P^T and P^T A = LU
print("P")
print(P)

print("U")
print(U)
print("L")
print(L)

print("sl.lu(A)")
print(sl.lu(A))

print('\nA = P @ L @ U? ',np.allclose(A_orig, P @ L @ U))


# a new A with swapped rows based on observation of P
A = np.array([[25. ,91. ,55. ,67.],
               [ 5.,  7.,  5.,  9.], 
               [20., 77., 41., 48.],
               [ 5., 14., 7.,  10.]])
print("A (permutated according to P")
print(A)

print("A = PLU --> P^T A = LU. Print P^T A (= above)")
print(P.T @ A_orig)

A original
[[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]
P
[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
U
[[ 25.          91.          55.          67.        ]
 [  0.         -11.2         -6.          -4.4       ]
 [  0.           0.          -5.25        -7.25      ]
 [  0.           0.           0.           0.66666667]]
L
[[ 1.          0.          0.          0.        ]
 [ 0.2         1.          0.          0.        ]
 [ 0.8        -0.375       1.          0.        ]
 [ 0.2         0.375       0.33333333  1.        ]]
sl.lu(A)
(array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]]), array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.2       ,  1.        ,  0.        ,  0.        ],
       [ 0.8       , -0.375     ,  1.        ,  0.        ],
       [ 0.2       ,  0.375     ,  0.33333333,  1.        ]]), array([[ 25.        ,  91.        ,  55.        ,  67.       

In [13]:
### Ex 6.3 implement partial pivoting (disregard b)

# Swap rows function from L5
def swap_row(A, i, j):
    """ Swap rows i and j of the matrix A.
    eg swap row 2 3 --> swap_row(A, 1, 2)
    if statement skips func if rows are the same
    """ 
    if i != j:
        print('\nswapping rows', i,'and', j)

    iA = np.copy(A[i, :])
    jA = np.copy(A[j, :])

    A[i, :] = jA
    A[j, :] = iA

    "no need for return statement as it directly changes the "

# LU decomp with partial pivoting
def LU_decomp_pp(A):
    """ A function to covert A into upper triangluar form through row operations.

    This version uses partial pivoting and ignores b.
    
    Note that A and b are overwritten, and hence we do not need to return anything
    from the function.
    """

    # check A is square and its number of rows and columns same as size of the vector b
    rows, cols = np.shape(A)
    assert(rows == cols)

    L = np.eye(rows)
    "P NOT P^T"
    PT = np.eye(rows)

    # Loop over each pivot row - all but the last row
    for k in range(rows-1):
        # Swap rows so we are always dividing through by the largest number.
        # initiatise kmax with the current pivot row (k)
        print("pre-swap", A)
        kmax = k
        # loop over all entries below the pivot and select the k with the largest abs value
        for j in range(k+1, rows):
            if abs(A[kmax, k]) < abs(A[j, k]):
                kmax = j
        # and swap the current PIVOT row (k) with the row with the largest abs value below the pivot
        swap_row(A, kmax, k)
        swap_row(PT, kmax, k)

        print("after-swap", A)
        for i in range(k+1, rows):
            "for EACH row below pivot row (i = row below pivot)"
            s = (A[i, k] / A[k, k])
            A[i, :] = A[i, :] - s * A[k, :]
            L[i, k] = s

    U = A
    "RETURN PT.T (= P)"
    "result been affected by row swapping --> P keeps track of this"
    "use the INVERSE of this to fix LU decomposition"
    return PT.T , L, U

A = np.array([[ 5., 7.,   5.,  9.],
              [ 5., 14.,  7., 10.],
              [20., 77., 41., 48.],
              [25., 91. ,55., 67.]])

A_orig = np.copy(A)

P, L, U = LU_decomp_pp(A)

print("A original")
print(A_orig)

print("P")
print(P)
print("U")
print(U)
print("L")
print(L)

print("A = P @ L @ U")
" L and U has been affected by row swapping as well --> use P to fix"
" s location changes from swapping --> messes up L"
print( P @ ( L @ U ) )

print('\nA = P @ L @ U? ',np.allclose(A_orig, P @ L @ U))

pre-swap [[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]

swapping rows 3 and 0

swapping rows 3 and 0
after-swap [[25. 91. 55. 67.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [ 5.  7.  5.  9.]]
pre-swap [[ 25.   91.   55.   67. ]
 [  0.   -4.2  -4.   -3.4]
 [  0.    4.2  -3.   -5.6]
 [  0.  -11.2  -6.   -4.4]]

swapping rows 3 and 1

swapping rows 3 and 1
after-swap [[ 25.   91.   55.   67. ]
 [  0.  -11.2  -6.   -4.4]
 [  0.    4.2  -3.   -5.6]
 [  0.   -4.2  -4.   -3.4]]
pre-swap [[ 25.    91.    55.    67.  ]
 [  0.   -11.2   -6.    -4.4 ]
 [  0.     0.    -5.25  -7.25]
 [  0.     0.    -1.75  -1.75]]
after-swap [[ 25.    91.    55.    67.  ]
 [  0.   -11.2   -6.    -4.4 ]
 [  0.     0.    -5.25  -7.25]
 [  0.     0.    -1.75  -1.75]]
A original
[[ 5.  7.  5.  9.]
 [ 5. 14.  7. 10.]
 [20. 77. 41. 48.]
 [25. 91. 55. 67.]]
P
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]
U
[[ 25.          91.          55.          67.        ]
 [  0.         -11